In [115]:
# On-the fly reloads of libraries
%load_ext autoreload
%autoreload 2

# Import libraries
import numpy as np
from astropy.io import fits
import matplotlib.pyplot as plt

# Local modules
from Visualization import plot_stellar
from StarDetection import extract_stars
from FieldAnalysis import get_image_area_physical, get_diagonal_distance, get_pixel_scale, deg_to_px
from Solver import solve_field
from IndexHandler import get_index
from Visualization import compare_hashes
from Utilities import gaia_asterism_to_coords, local_asterism_to_coords
from astropy.wcs.utils import fit_wcs_from_points
import astropy.units as u

# Import astropy skycoord
from astropy.coordinates import SkyCoord

# Parameters
from dotenv import load_dotenv
import os
load_dotenv()
FIELD_DENSITY = float(os.getenv("FIELD_DENSITY"))
PIXEL_SCALE = float(os.getenv("PIXEL_SCALE"))

# Plotting setting
import PyQt5
%matplotlib qt

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Get data

In [6]:
helix = fits.getdata('../Data/Helix.fits')
helixhdul = fits.open('../Data/Helix.fits')
plot_stellar(helix,stretch=False,vlims=[10,98])

(<Figure size 3420x1888 with 1 Axes>, <Axes: >)

### Examine the field

In [7]:
# Get field area
field_area = get_image_area_physical(fits.open('../Data/Helix.fits'))
print('Field area: {:.2f} square degrees'.format(field_area))

# This corresponds to the number of stars in the field
n_stars = FIELD_DENSITY * field_area
print('Utilized number of stars: {:.0f}'.format(n_stars))

pixel_scale = get_pixel_scale()
print('Pixel scale: {:.8f} deg/px'.format(pixel_scale))

print(f"Field density: {FIELD_DENSITY} stars/sq.deg")

Field area: 0.28 square degrees
Utilized number of stars: 28
Pixel scale: 0.00012917 deg/px
Field density: 100.0 stars/sq.deg


### Run star extraction

In [8]:
stars = extract_stars(helix)

In [9]:
# Plot the stars
fig,ax = plot_stellar(helix,stretch=True,vlims=[3,99.75])
for star in stars:
    ax.add_artist(plt.Circle((star['x'],star['y']),radius=np.median(star['fwhm'])*1.5,color='r',fill=False))

# Highlight the n_stars brightest stars
brightest_stars = stars[np.argsort(stars['flux'])[-int(n_stars):]]
ax.scatter(brightest_stars['x'],brightest_stars['y'],marker='x',color='g',s=10)

In [12]:
import numpy as np
import matplotlib.pyplot as plt

# Assuming you have an array of standard deviations
std_deviations = stars['fwhm'] / (2 * np.sqrt(2 * np.log(2))) * pixel_scale * 3600 # Arcseconds per deg
mean_fwhm = np.mean(stars['fwhm']) * pixel_scale * 3600

# Generate some x values
x_values = np.linspace(0.75, 1.65, 1000)

# Calculate the mean and standard error of the standard deviations
mean_std = np.mean(std_deviations)
std_error = np.std(std_deviations, ddof=1) / np.sqrt(len(std_deviations))  # Standard error

# Calculate the upper and lower bounds for the 1 sigma error band with propagated error
upper_bound = (1 / (np.sqrt(2 * np.pi) * mean_std)) * np.exp(-0.5 * ((x_values - mean_std) / (std_error + std_deviations.std())) ** 2)
lower_bound = (1 / (np.sqrt(2 * np.pi) * mean_std)) * np.exp(-0.5 * ((x_values - mean_std) / (std_error - std_deviations.std())) ** 2)

# Calculate the mean of upper and lower bounds to ensure symmetry around the mean Gaussian
mean_upper_lower = (upper_bound + lower_bound) / 2

# Plotting
fig,ax = plt.subplots(tight_layout=True)
ax.plot(x_values, mean_upper_lower, color='k', label='Mean Gaussian')
ax.fill_between(x_values, upper_bound, lower_bound, alpha=0.5, label='1 sigma error band')
ax.legend()
ax.set_xlabel(r'$\sigma$ (Arcseconds)')
ax.set_ylabel('Probability density')
ax.set_title('Mean stellar profile')
# Highlight FWHM

Text(0.5, 1.0, 'Mean stellar profile')

### Let's try to solve the field

In [18]:
gaia_stars = get_index(helixhdul,337.3,-20.8)

Checking cache entries ...
Found relevant cache entry: ./GAIACache/337.30_-20.80_FD100.00_SRM2.00.votable.gz.
Using cached GAIA results.


#### Build asterims of input image

In [21]:
from AsterismBuilder import build_asterisms_from_input_image
from AsterismBuilder import build_asterisms_from_GAIA

asterisms_GAIA,hashes_GAIA = build_asterisms_from_GAIA(gaia_stars,visualize=True)
asterisms,hashes_IMAGE = build_asterisms_from_input_image(helixhdul,visualize=True)

In [22]:
# Compare asterism to hash
fig,ax = plt.subplots(1,2,figsize=(10,5),tight_layout=True)
asterism = asterisms[0]
hash = hashes_IMAGE[0]
xs = np.array([asterism['xa'],asterism['xc'], asterism['xd'], asterism['xb']])
ys = np.array([asterism['ya'],asterism['yc'], asterism['yd'], asterism['yb']])

xs_hash = np.array([0,hash['xc'],hash['xd'],hash['xb']])
ys_hash = np.array([0,hash['yc'],hash['yd'],hash['yb']])

ax[0].scatter(xs,ys,color='r')
ax[0].set_xlabel('x [px]')
ax[0].set_ylabel('y [px]')

ax[1].scatter(xs_hash,ys_hash,color='b')
ax[1].set_xlabel('x [deg]')
ax[1].set_ylabel('y [deg]')

order = ['a','c','d','b']

for (x,y,letter) in zip(xs,ys,order):
    ax[0].annotate(letter,(x,y),color='r',fontsize=20)
    
for (x,y,letter) in zip(xs_hash,ys_hash,order):
    ax[1].annotate(letter,(x,y),color='b',fontsize=20)

### With the full solve_field code:

Checking cache entries ...
Found relevant cache entry: ./GAIACache/337.30_-20.80_FD100.00_SRM2.00.votable.gz.
Using cached GAIA results.


In [131]:
order = ['a','c','d','b']

fig,ax = plt.subplots(1,2,figsize=(10,5),tight_layout=True)
ax[0].scatter(gaia_stars['ra'],gaia_stars['dec'],color='r')
gaia_xs = np.array([gaia_asterism['xa'],gaia_asterism['xc'], gaia_asterism['xd'], gaia_asterism['xb']])
gaia_ys = np.array([gaia_asterism['ya'],gaia_asterism['yc'], gaia_asterism['yd'], gaia_asterism['yb']])
ax[0].scatter(gaia_xs,gaia_ys,color='b')
for (x,y,letter) in zip(gaia_xs,gaia_ys,order):
    ax[0].annotate(letter,(x,y),color='b',fontsize=20)

local_xs = np.array([local_asterism['xa'],local_asterism['xc'], local_asterism['xd'], local_asterism['xb']])
local_ys = np.array([local_asterism['ya'],local_asterism['yc'], local_asterism['yd'], local_asterism['yb']])
imgdata = helix
vlims = np.percentile(imgdata,[1,99])
ax[1].imshow(imgdata,origin='lower',cmap='gray_r',vmin=vlims[0],vmax=vlims[1])
ax[1].scatter(local_xs,local_ys,color='b')
ax[1].set_xlim(0,4096)
ax[1].set_ylim(0,4096)
for (x,y,letter) in zip(local_xs,local_ys,order):
    ax[1].annotate(letter,(x,y),color='b',fontsize=20)

In [132]:
compare_hashes(hash_gaia,hash_local)

No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


In [133]:
wcs,score,gaia_asterism,local_asterism,hash_gaia,hash_local,hashnormdiffs = solve_field(helixhdul,337.3,-20.8)

gaia_stars_to_coords = wcs.world_to_pixel_values(gaia_stars['ra'],gaia_stars['dec'])

# Plot the gaia stars on the image
fig,ax = plt.subplots(figsize=(5,5),tight_layout=True)
ax.imshow(helix,origin='lower',cmap='gray_r',vmin=vlims[0],vmax=vlims[1])
ax.scatter(gaia_stars_to_coords[0],gaia_stars_to_coords[1],color='r')

for (x,y,letter) in zip(gaia_xs,gaia_ys,order):
    x,y = wcs.world_to_pixel_values(x,y)
    ax.scatter(x,y,color='g')
    ax.annotate(letter,(x,y),color='g',fontsize=20)
    
for (x,y,letter) in zip(local_xs,local_ys,order):
    ax.scatter(x,y,color='b')
    ax.annotate(letter,(x,y),color='b',fontsize=20)
    
#ax.set_xlim(0,4096)
#ax.set_ylim(0,4096)

Checking cache entries ...
Found relevant cache entry: ./GAIACache/337.30_-20.80_FD100.00_SRM2.00.votable.gz.
Using cached GAIA results.


### Work